In [7]:
import requests
import pandas as pd
import json
import time
import re
import numpy as np
import spacy
import psycopg2
# apache-airflow[all]
# apache-airflow[postgres]

In [8]:
info_vac_v1 = pd.DataFrame()

for i in range(0,2):
    
    params = {
        'text': '!(аналитик данных OR data analyst OR бизнес-аналитик OR BI-аналитик or data engineer)', # Текст фильтра
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': i, # Индекс страницы поиска на HH
        'per_page': 10 # Кол-во вакансий на 1 странице
        }
    
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    time.sleep(0.5)
    req.close()
    json_data = json.loads(data)
    new_data = pd.DataFrame(json_data['items']).reset_index(drop=True)
    info_vac_v1 = pd.concat([info_vac_v1, new_data], ignore_index=True)

In [9]:

salary = pd.json_normalize(info_vac_v1['salary'])
address = pd.json_normalize(info_vac_v1['address'])

address = address.drop(columns=['id'])

employer = pd.json_normalize(info_vac_v1['employer'])

employer_to_drop = ['logo_urls.90', 'logo_urls.240', 'url', 'alternate_url', 'vacancies_url', 'logo_urls.original', 'logo_urls']
employer_exist_to_drop = [col for col in employer_to_drop if col in employer.columns]
employer = employer.drop(columns=employer_exist_to_drop)
employer = employer.rename(columns={'id': 'id_employer', "name": "name_employer"})

snippet = pd.json_normalize(info_vac_v1['snippet'])

schedule = pd.json_normalize(info_vac_v1['schedule'])
schedule.drop(columns=['id'], inplace=True)
schedule.rename(columns={'name': 'schedules'}, inplace=True)

def sep_list_columns(list):
    length = len(list)
    if length >= 1:
        return list[0]

work_format = pd.json_normalize(info_vac_v1['work_format'].apply(sep_list_columns))
work_format.rename(columns={'name': 'format_works'}, inplace=True)

working_hours = pd.json_normalize(info_vac_v1['working_hours'].apply(sep_list_columns))
working_hours.rename(columns={'name': 'hours_working'}, inplace=True)

work_schedule_by_days = pd.json_normalize(info_vac_v1['work_schedule_by_days'].apply(sep_list_columns))
work_schedule_by_days.rename(columns={'name': 'schedule_work_by_days'}, inplace=True)

professional_roles = pd.json_normalize(info_vac_v1['professional_roles'].apply(sep_list_columns))
professional_roles.rename(columns={'name': 'roles_professional'}, inplace=True)

experience = pd.json_normalize(info_vac_v1['experience'])
experience = experience.drop(columns=['id'])
experience = experience.rename(columns={'name': 'experience_'})

employment = pd.json_normalize(info_vac_v1['employment'])
employment = employment.drop(columns=['id'])
employment = employment.rename(columns={'name': 'employment_'})

info_vac_v1 = pd.concat([info_vac_v1, salary, address, employer, snippet, schedule, work_format['format_works'], working_hours['hours_working'], work_schedule_by_days['schedule_work_by_days'], professional_roles['roles_professional'], experience['experience_'], employment['employment_']], axis = 1)

columns_to_drop = [
    'area', 'salary', 'employment_form', 'salary_range', 'type', 'address', 
    'employer', 'contacts', 'snippet', 'schedule', 'work_format', 
    'employment_form', 'working_hours', 'work_schedule_by_days',
    'professional_roles', 'experience', 'employment', 'metro_stations', 
    'branding', 'department', 'description', 'insider_interview', 
    'response_url', 'sort_point_distance', 'adv_response_url', 'adv_context',
    'metro', 'relations', 'working_days', 'working_time_intervals', 
    'working_time_modes', 'fly_in_fly_out_duration', 'night_shifts', 
    'is_adv_vacancy', 'archived', 'show_logo_in_search', 'show_contacts', 
    'brand_snippet', 'created_at', 'url', 'accept_temporary', 
    'accept_incomplete_resumes', 'internship', 'metro.station_id', 
    'metro.line_id', 'metro.lat', 'metro.lng'
]

existing_columns_to_drop = [col for col in columns_to_drop if col in info_vac_v1.columns]

info_vac_v1 = info_vac_v1.drop(columns=existing_columns_to_drop)

exchange_rates = {
    'USD': 80.0,  # 1 USD = 80 RUB
    'EUR': 90.0,  # 1 EUR = 90 RUB
    'RUR': 1.0     # 1 RUB = 1 RUB
}

def check_salary(row):

    if pd.isna(row['to']) and pd.isna(row['from']):
        return np.nan
    elif pd.isna(row['to']):
        total_salary = row['from']
    elif pd.isna(row['from']):
        total_salary = row['to']
    else:
        total_salary = (row['from'] + row['to']) / 2

    currency = row['currency']
    if pd.notna(currency):
        total_salary *= exchange_rates.get(currency, 1.0) # Конвертация в рубли
    if pd.notna(row['gross']) and row['gross'] == True:
        total_salary *= 0.87 # Вычет налогов
    return total_salary

info_vac_v1['total_salary'] = info_vac_v1.apply(check_salary, axis = 1)

In [23]:
info_vac_v1

,id,premium,name,has_test,response_letter_required,published_at,apply_alternate_url,alternate_url,from,to,...,requirement,responsibility,schedules,format_works,hours_working,schedule_work_by_days,roles_professional,experience_,employment_,total_salary
0,122702826,False,Fullstack аналитик,False,False,2025-07-11T11:48:33+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/122702826,NaN,NaN,...,Компетенции в проработке пользовательских сцен...,"Выявлять, анализировать и создавать бизнес-тре...",Удаленная работа,Удалённо,8 часов,5/2,Аналитик,От 3 до 6 лет,Полная занятость,NaN
1,123440309,False,Бизнес-аналитик,True,False,2025-07-30T17:36:04+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123440309,NaN,NaN,...,Опыт работы в бизнес-анализе 1-3 года. Опыт мо...,Выявление корневых бизнес-потребностей. Оценка...,Удаленная работа,Удалённо,8 часов,5/2,Бизнес-аналитик,От 1 года до 3 лет,Полная занятость,NaN
2,123436009,False,Стажер аналитик,True,False,2025-07-30T16:16:01+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123436009,NaN,NaN,...,Ты будешь: - Продвинутое владение Excel (форму...,Готовить отчеты по доходности и маржинальности...,Полный день,На месте работодателя,8 часов,5/2,Другое,Нет опыта,Полная занятость,NaN
3,121163840,False,"Бизнес-аналитик CJE/CJM в стрим ""Устройства са...",False,False,2025-07-22T16:15:45+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/121163840,NaN,NaN,...,Успешный опыт участия в проекте с большим коли...,Проработка бизнес-процессов и клиентских путей...,Полный день,На месте работодателя,8 часов,5/2,Бизнес-аналитик,От 1 года до 3 лет,Полная занятость,NaN
4,123465383,False,Аналитик,False,False,2025-07-31T12:55:11+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123465383,NaN,NaN,...,"Сильными аналитическими навыки, стратегическим...","Сбор, анализ и подготовка информации для дальн...",Полный день,На месте работодателя,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN
5,123464963,False,Бизнес-аналитик,False,False,2025-07-31T12:46:30+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123464963,NaN,NaN,...,Высшее образование/дополнительное профессионал...,"Анализ функциональных требований, user stories...",Полный день,На месте работодателя,8 часов,5/2,Бизнес-аналитик,От 3 до 6 лет,Полная занятость,NaN
6,123391639,False,Системный аналитик,False,False,2025-07-29T17:02:04+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123391639,NaN,NaN,...,Знание SQL и баз <highlighttext>данных</highli...,"Определение связей между подсистемами, влияние...",Полный день,Гибрид,8 часов,5/2,Системный аналитик,От 1 года до 3 лет,Полная занятость,NaN
7,123362863,False,Аналитик данных,False,False,2025-07-29T12:29:18+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123362863,NaN,NaN,...,Опыт работы с большими данными и сложными стру...,Развитие и создание зон роста компании через д...,Полный день,На месте работодателя,8 часов,5/2,"BI-аналитик, аналитик данных",От 1 года до 3 лет,Полная занятость,NaN
8,123362968,False,Бизнес-аналитик,False,False,2025-07-29T12:31:02+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123362968,NaN,NaN,...,Опыт работы бизнес-аналитиком в проектных/прод...,"Сбор, агрегация и анализ требований и материал...",Полный день,На месте работодателя,8 часов,5/2,Бизнес-аналитик,От 1 года до 3 лет,Полная занятость,NaN
9,123412522,False,Бизнес-аналитик,True,False,2025-07-30T10:46:57+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123412522,NaN,NaN,...,Опыт работы аналитиком/продуктовым аналитиком ...,Поддержка продакта во встречах и коммуникации ...,Полный день,На месте работодателя,8 часов,5/2,Бизнес-аналитик,От 1 года до 3 лет,Полная занятость,NaN


In [10]:
description_info = []
for id in info_vac_v1['id']:
    try:
        # Делаем запрос к API
        req_vac = requests.get(f'https://api.hh.ru/vacancies/{id}') 
        req_vac.raise_for_status() 
        data = req_vac.content.decode() 
        req_vac.close()
        
        time.sleep(0.4)  # Задержка между запросами
        
        data_vac = json.loads(data)
        
        # Обработка skills с проверкой наличия ключа
        skills = []
        if 'key_skills' in data_vac and isinstance(data_vac['key_skills'], list):
            skills = [i['name'] for i in data_vac['key_skills']]
        
        description_info.append({
            'vacancy_id': id,
            'description': data_vac.get('description', ''), 
            'key_skills_under_desc': skills,
        })
    except Exception as e:
        print(f"Неожиданная ошибка для вакансии {id}: {e}")
        continue
        

In [11]:
df_description_info = pd.DataFrame(description_info)
info_vac_merge = info_vac_v1.merge(df_description_info, left_on='id', right_on='vacancy_id', how='left').drop(columns = ['vacancy_id'])

info_vac_merge.rename(columns={'description_y': 'description',
                         'key_skills_under_desc_y': 'key_skills_under_desc'}, inplace=True)
info_vac_merge.to_csv('info_vac.csv')

In [12]:
skills_keywords = [
    # Языки программирования
    'SQL', 'Python', 'R', 'Scala', 'Java', 'C#', 'Julia', '1C',
    
    # BI и визуализация
    'Power BI', 'Tableau', 'Qlik', 'Looker', 'Metabase', 'Redash', 'Superset',
    'Excel', 'Google Sheets', 'Data Studio', 'Plotly', 'Matplotlib', 'Seaborn',
    
    # Базы данных
    'PostgreSQL', 'MySQL', 'MS SQL', 'Oracle', 'ClickHouse', 'Greenplum',
    'MongoDB', 'Redis', 'Cassandra', 'Snowflake', 'BigQuery', 'Redshift',
    
    # Big Data
    'Hadoop', 'Spark', 'Hive', 'Kafka', 'Airflow', 'Flink', 'Databricks',
    
    # Облачные платформы
    'AWS', 'Azure', 'GCP', 'Yandex Cloud', 'IBM Cloud',
    
    # ETL и обработка данных
    'DWH', 'ETL', 'ELT', 'Data Vault', 'Data Lake', 'Data Mesh',
    'Informatica', 'Talend', 'SSIS', 'Alteryx', 'dbt', 'Apache NiFi',
    
    # Аналитика
    'Machine Learning', 'ML', 'AI', 'Deep Learning', 'NLP', 'Computer Vision',
    'Statistics', 'A/B тестирование', 'Predictive Modeling', 'Time Series',
    'EDA', 'Feature Engineering', 'MLflow', 'Kubeflow',
    
    # Управление
    'Scrum', 'Agile', 'Kanban', 'Jira', 'Confluence', 'Git', 'CI/CD',
    
    # Дополнительные технологии
    'Docker', 'Kubernetes', 'Linux', 'Bash', 'Pandas', 'NumPy', 'SciPy',
    'Scikit-learn', 'TensorFlow', 'PyTorch', 'Keras', 'XGBoost', 'CatBoost',
    'LightGBM', 'OpenCV', 'NLTK', 'spaCy', 'Hugging Face', 'REST', 'API', 'request', 'WebSocket',
    
    # Математика
    'Математика', 'Статистика', 'Математический анализ', 'Линейная алгебра', 
    'Теория Вероятностей', 'Дискретная математика']


def extract_skills(desc):
    found_skills = []
    desc = str(desc).lower()
    for skill in skills_keywords:
        if re.search(r'\b' + re.escape(skill.lower()) + r'\b', desc):
            found_skills.append(skill)
    return found_skills

nlp = spacy.load("ru_core_news_sm")
def check_intern_nlp(desc):
    
    doc = nlp(desc.lower()) 

    internship_lemmas = {
        "интерн", "стажёр", 
        "intern", "trainee" 
    }
    for token in doc:
        # Проверяем лемму (начальную форму) слова
        if token.lemma_ in internship_lemmas:
            return True
            
    return False

info_vac_merge = pd.read_csv('info_vac.csv', index_col=0)

info_vac_merge.columns = map(lambda x: x.replace('.', '_'), info_vac_merge.columns.to_list())

info_vac_merge['skills_in_desc'] = info_vac_merge['description'].apply(extract_skills)

info_vac_merge = info_vac_merge[info_vac_merge['id'].notna()]
info_vac_merge['is_intern'] = info_vac_merge['name'].apply(check_intern_nlp)
info_vac_merge = info_vac_merge[info_vac_merge['id_employer'].notna()]
info_vac_merge['published_at'] = pd.to_datetime(info_vac_merge['published_at'], utc = True)
info_vac_merge.rename(columns={'from': 'from_salary',
                               'to': 'to_salary'}, inplace=True)

skills_desc = info_vac_merge['skills_in_desc'].explode('skills_in_desc')
skills_counts = skills_desc.value_counts()
vacancy_skills = info_vac_merge[['id', 'skills_in_desc']].explode('skills_in_desc')

employer = info_vac_merge[['id_employer', 'name_employer', 'accredited_it_employer', 'trusted']].drop_duplicates(subset = ['id_employer'], keep = 'first')
employer = employer[employer['id_employer'].notna()]

info_vac_merge.drop(columns=['name_employer', 'accredited_it_employer', 'trusted'], inplace=True)

In [19]:
DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'postgres',
    'user': 'postgres',
    'password': '123'
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS employers (
              id_employer INTEGER PRIMARY KEY,
              name_employer VARCHAR(200),
              accredited_it_employer TEXT,
              trusted BOOLEAN);
          """)
        
        columns = ', '.join(employer.columns)
        placeholders = ', '.join(['%s'] * len(employer.columns))
        for _, row in employer.iterrows():
            cursor.execute(f"INSERT INTO employers ({columns}) "
                            f"VALUES ({placeholders})"
                            f"ON CONFLICT (id_employer) DO NOTHING", tuple(row))
        conn.commit()
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS vacancy (
              id INTEGER PRIMARY KEY,
              premium BOOLEAN,
              name TEXT,
              has_test BOOLEAN,
              response_letter_required BOOLEAN,
              published_at TIMESTAMP WITH TIME ZONE,
              apply_alternate_url TEXT,
              alternate_url TEXT,
              from_salary NUMERIC(10,2),
              to_salary NUMERIC(10,2),
              currency VARCHAR(20),
              gross TEXT,
              city TEXT,
              street TEXT,
              building TEXT,
              lat NUMERIC(10,6),
              lng NUMERIC(10,6),
              raw TEXT,
              metro_station_name TEXT,
              metro_line_name TEXT,
              id_employer INTEGER REFERENCES employers(id_employer),
              requirement TEXT,
              responsibility TEXT,
              schedules TEXT,
              format_works TEXT, 
              hours_working TEXT,
              schedule_work_by_days TEXT,
              roles_professional TEXT,
              experience_ TEXT,
              employment_ TEXT, 
              total_salary NUMERIC(10,2),
              description TEXT, 
              key_skills_under_desc TEXT,
              skills_in_desc TEXT,
              is_intern BOOLEAN   
              );
          """)
        
        columns = ', '.join(info_vac_merge.columns)
        placeholders = ', '.join(['%s'] * len(info_vac_merge.columns))
        for _, row in info_vac_merge.iterrows():
            cursor.execute(f"INSERT INTO vacancy ({columns})"
                            f"VALUES ({placeholders})"
                            f"ON CONFLICT (id) DO NOTHING", tuple(row))
        conn.commit()
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS skills (
              id INTEGER REFERENCES vacancy(id),
              skills_in_desc TEXT
              );
          """)
        
        columns = ', '.join(vacancy_skills.columns)
        placeholders = ', '.join(['%s'] * len(vacancy_skills.columns))
        for _, row in vacancy_skills.iterrows():
            cursor.execute("DELETE FROM skills WHERE id = %s", (row['id'],))
            cursor.execute(f"INSERT INTO skills ({columns})"
                            f"VALUES ({placeholders})", tuple(row))
        conn.commit()
        
        

In [6]:
import requests
import pandas as pd
import json
import time
import re
import numpy as np
import spacy


nlp = spacy.load("ru_core_news_sm")
# Выгрузка и обработка основной информации по вакансиям
def init_hh_main_data():

    info_vac_v1 = pd.DataFrame()

    for i in range(0, 10):
        params = {
            'text': '!(аналитик данных OR data analyst OR бизнес-аналитик OR BI-аналитик or data engineer)',
            # Текст фильтра
            'area': 1,  # Поиск ощуществляется по вакансиям города Москва
            'page': i,  # Индекс страницы поиска на HH
            'per_page': 100  # Кол-во вакансий на 1 странице
        }

        req = requests.get('https://api.hh.ru/vacancies', params)  # Посылаем запрос к API
        data = req.content.decode()  # Декодируем его ответ, чтобы Кириллица отображалась корректно
        time.sleep(0.5)
        req.close()
        json_data = json.loads(data)
        new_data = pd.DataFrame(json_data['items']).reset_index(drop=True)
        info_vac_v1 = pd.concat([info_vac_v1, new_data], ignore_index=True)

    # Обработка данных
    salary = pd.json_normalize(info_vac_v1['salary'])

    address = pd.json_normalize(info_vac_v1['address'])
    address = address.drop(columns=['id'])

    employer = pd.json_normalize(info_vac_v1['employer'])
    employer_to_drop = ['logo_urls.90', 'logo_urls.240', 'url', 'alternate_url', 'vacancies_url', 'logo_urls.original',
                        'logo_urls']
    employer_exist_to_drop = [col for col in employer_to_drop if col in employer.columns]
    employer = employer.drop(columns=employer_exist_to_drop)
    employer = employer.rename(columns={'id': 'id_employer', "name": "name_employer"})

    snippet = pd.json_normalize(info_vac_v1['snippet'])

    schedule = pd.json_normalize(info_vac_v1['schedule'])
    schedule.drop(columns=['id'], inplace=True)
    schedule.rename(columns={'name': 'schedules'}, inplace=True)

    def sep_list_columns(list):
        length = len(list)
        if length >= 1:
            return list[0]

    work_format = pd.json_normalize(info_vac_v1['work_format'].apply(sep_list_columns))
    work_format.rename(columns={'name': 'format_works'}, inplace=True)

    working_hours = pd.json_normalize(info_vac_v1['working_hours'].apply(sep_list_columns))
    working_hours.rename(columns={'name': 'hours_working'}, inplace=True)

    work_schedule_by_days = pd.json_normalize(info_vac_v1['work_schedule_by_days'].apply(sep_list_columns))
    work_schedule_by_days.rename(columns={'name': 'schedule_work_by_days'}, inplace=True)

    professional_roles = pd.json_normalize(info_vac_v1['professional_roles'].apply(sep_list_columns))
    professional_roles.rename(columns={'name': 'roles_professional'}, inplace=True)

    experience = pd.json_normalize(info_vac_v1['experience'])
    experience = experience.drop(columns=['id'])
    experience = experience.rename(columns={'name': 'experience_'})

    employment = pd.json_normalize(info_vac_v1['employment'])
    employment = employment.drop(columns=['id'])
    employment = employment.rename(columns={'name': 'employment_'})

    info_vac_v1 = pd.concat([info_vac_v1, salary, address, employer, snippet, schedule, work_format['format_works'],
                             working_hours['hours_working'], work_schedule_by_days['schedule_work_by_days'],
                             professional_roles['roles_professional'], experience['experience_'],
                             employment['employment_']], axis=1)
    columns_to_drop = [
        'area', 'salary', 'employment_form', 'salary_range', 'type', 'address',
        'employer', 'contacts', 'snippet', 'schedule', 'work_format',
        'employment_form', 'working_hours', 'work_schedule_by_days',
        'professional_roles', 'experience', 'employment', 'metro_stations',
        'branding', 'department', 'description', 'insider_interview',
        'response_url', 'sort_point_distance', 'adv_response_url', 'adv_context',
        'metro', 'relations', 'working_days', 'working_time_intervals',
        'working_time_modes', 'fly_in_fly_out_duration', 'night_shifts',
        'is_adv_vacancy', 'archived', 'show_logo_in_search', 'show_contacts',
        'brand_snippet', 'created_at', 'url', 'accept_temporary',
        'accept_incomplete_resumes', 'internship', 'metro.station_id',
        'metro.line_id', 'metro.lat', 'metro.lng'
    ]
    existing_columns_to_drop = [col for col in columns_to_drop if col in info_vac_v1.columns]
    info_vac_v1 = info_vac_v1.drop(columns=existing_columns_to_drop)
    exchange_rates = {
        'USD': 80.0,  # 1 USD = 80 RUB
        'EUR': 90.0,  # 1 EUR = 90 RUB
        'RUR': 1.0  # 1 RUB = 1 RUB
    }
    # Обработка зарплаты
    def check_salary(row):
        if pd.isna(row['to']) and pd.isna(row['from']):
            return np.nan
        elif pd.isna(row['to']):
            total_salary = row['from']
        elif pd.isna(row['from']):
            total_salary = row['to']
        else:
            total_salary = (row['from'] + row['to']) / 2
        currency = row['currency']
        if pd.notna(currency):
            total_salary *= exchange_rates.get(currency, 1.0)  # Конвертация в рубли
        if pd.notna(row['gross']) and row['gross'] == True:
            total_salary *= 0.87  # Вычет налогов
        return total_salary
    info_vac_v1['total_salary'] = info_vac_v1.apply(check_salary, axis=1)

    return info_vac_v1

# Выгрузка по каждой вакансии
def load_info_vac(info_vac_v1):
    description_info = []
    for id in info_vac_v1['id']:
        try:
            # Делаем запрос к API
            req_vac = requests.get(f'https://api.hh.ru/vacancies/{id}')
            req_vac.raise_for_status()
            data = req_vac.content.decode()
            req_vac.close()

            time.sleep(0.4)  # Задержка между запросами

            data_vac = json.loads(data)

            # Обработка skills с проверкой наличия ключа
            skills = []
            if 'key_skills' in data_vac and isinstance(data_vac['key_skills'], list):
                skills = [i['name'] for i in data_vac['key_skills']]

            description_info.append({
                'vacancy_id': id,
                'description': data_vac.get('description', ''),
                'key_skills_under_desc': skills,
            })
        except Exception as e:
            print(f"Неожиданная ошибка для вакансии {id}: {e}")
            continue

    return description_info

# Из текста вакансий выносим инфу по скилам
def extract_skills(desc):
    skills_keywords = [
        # Языки программирования
        'SQL', 'Python', 'R', 'Scala', 'Java', 'C#', 'Julia', '1C',

        # BI и визуализация
        'Power BI', 'Tableau', 'Qlik', 'Looker', 'Metabase', 'Redash', 'Superset',
        'Excel', 'Google Sheets', 'Data Studio', 'Plotly', 'Matplotlib', 'Seaborn',

        # Базы данных
        'PostgreSQL', 'MySQL', 'MS SQL', 'Oracle', 'ClickHouse', 'Greenplum',
        'MongoDB', 'Redis', 'Cassandra', 'Snowflake', 'BigQuery', 'Redshift',

        # Big Data
        'Hadoop', 'Spark', 'Hive', 'Kafka', 'Airflow', 'Flink', 'Databricks',

        # Облачные платформы
        'AWS', 'Azure', 'GCP', 'Yandex Cloud', 'IBM Cloud',

        # ETL и обработка данных
        'DWH', 'ETL', 'ELT', 'Data Vault', 'Data Lake', 'Data Mesh',
        'Informatica', 'Talend', 'SSIS', 'Alteryx', 'dbt', 'Apache NiFi',

        # Аналитика
        'Machine Learning', 'ML', 'AI', 'Deep Learning', 'NLP', 'Computer Vision',
        'Statistics', 'A/B тестирование', 'Predictive Modeling', 'Time Series',
        'EDA', 'Feature Engineering', 'MLflow', 'Kubeflow',

        # Управление
        'Scrum', 'Agile', 'Kanban', 'Jira', 'Confluence', 'Git', 'CI/CD',

        # Дополнительные технологии
        'Docker', 'Kubernetes', 'Linux', 'Bash', 'Pandas', 'NumPy', 'SciPy',
        'Scikit-learn', 'TensorFlow', 'PyTorch', 'Keras', 'XGBoost', 'CatBoost',
        'LightGBM', 'OpenCV', 'NLTK', 'spaCy', 'Hugging Face', 'REST', 'API', 'request', 'WebSocket',

        # Математика
        'Математика', 'Статистика', 'Математический анализ', 'Линейная алгебра',
        'Теория Вероятностей', 'Дискретная математика']

    found_skills = []
    desc = str(desc).lower()
    for skill in skills_keywords:
        if re.search(r'\b' + re.escape(skill.lower()) + r'\b', desc):
            found_skills.append(skill)
    return found_skills

# Проверяем на возможную стажировку
def check_intern_nlp(desc, nlp_func):
    doc = nlp_func(desc.lower())

    internship_lemmas = {
        "интерн", "стажёр",
        "intern", "trainee"
    }
    for token in doc:
        # Проверяем лемму (начальную форму) слова
        if token.lemma_ in internship_lemmas:
            return True

    return False

def create_hh_vac_data_merge(hh_main_data, hh_vac_data):

    df_hh_vac_data = pd.DataFrame(hh_vac_data)
    hh_vac_data_merge = hh_main_data.merge(df_hh_vac_data, left_on='id', right_on='vacancy_id', how='left').drop(
        columns=['vacancy_id'])

    hh_vac_data_merge.rename(columns={'description_y': 'description',
                                      'key_skills_under_desc_y': 'key_skills_under_desc'}, inplace=True)

    hh_vac_data_merge.columns = map(lambda x: x.replace('.', '_'), hh_vac_data_merge.columns.to_list())

    hh_vac_data_merge['skills_in_desc'] = hh_vac_data_merge['description'].apply(extract_skills)
    hh_vac_data_merge = hh_vac_data_merge[hh_vac_data_merge['id'].notna()]

    hh_vac_data_merge['is_intern'] = hh_vac_data_merge['name'].apply(lambda x: check_intern_nlp(x, nlp))
    hh_vac_data_merge = hh_vac_data_merge[hh_vac_data_merge['id_employer'].notna()]

    hh_vac_data_merge['published_at'] = pd.to_datetime(hh_vac_data_merge['published_at'], utc=True)
    hh_vac_data_merge.rename(columns={'from': 'from_salary',
                                      'to': 'to_salary'}, inplace=True)
    hh_vac_data_merge[['id', 'id_employer']] = hh_vac_data_merge[['id', 'id_employer']].astype(int)

    return hh_vac_data_merge

def load_skills(hh_vac_data_merge):

    return hh_vac_data_merge[['id', 'skills_in_desc']].explode('skills_in_desc')

def load_employers(hh_vac_data_merge):

    employer = hh_vac_data_merge[['id_employer', 'name_employer', 'accredited_it_employer', 'trusted']].drop_duplicates(
        subset=['id_employer'], keep='first')
    employer = employer[employer['id_employer'].notna()]
    return employer

def load_vacancy(hh_vac_data_merge):
    hh_vac_data_merge.drop(columns=['name_employer', 'accredited_it_employer', 'trusted'], inplace=True)
    return hh_vac_data_merge


a = init_hh_main_data()
vacancy_1 = load_vacancy(create_hh_vac_data_merge(a, load_info_vac(a)))
employers_1 = load_employers(create_hh_vac_data_merge(a, load_info_vac(a)))
skills_1 = load_skills(create_hh_vac_data_merge(a, load_info_vac(a)))

DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'postgres',
    'user': 'postgres',
    'password': '123'
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS employers (
              id_employer INTEGER PRIMARY KEY,
              name_employer VARCHAR(200),
              accredited_it_employer TEXT,
              trusted BOOLEAN);
          """)
        
        columns = ', '.join(employers_1.columns)
        placeholders = ', '.join(['%s'] * len(employers_1.columns))
        for _, row in employers_1.iterrows():
            cursor.execute(f"INSERT INTO employers ({columns}) "
                            f"VALUES ({placeholders})"
                            f"ON CONFLICT (id_employer) DO NOTHING", tuple(row))
        conn.commit()
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS vacancy (
              id INTEGER PRIMARY KEY,
              premium BOOLEAN,
              name TEXT,
              has_test BOOLEAN,
              response_letter_required BOOLEAN,
              published_at TIMESTAMP WITH TIME ZONE,
              apply_alternate_url TEXT,
              alternate_url TEXT,
              from_salary NUMERIC(10,2),
              to_salary NUMERIC(10,2),
              currency VARCHAR(20),
              gross TEXT,
              city TEXT,
              street TEXT,
              building TEXT,
              lat NUMERIC(10,6),
              lng NUMERIC(10,6),
              raw TEXT,
              metro_station_name TEXT,
              metro_line_name TEXT,
              id_employer INTEGER REFERENCES employers(id_employer),
              requirement TEXT,
              responsibility TEXT,
              schedules TEXT,
              format_works TEXT, 
              hours_working TEXT,
              schedule_work_by_days TEXT,
              roles_professional TEXT,
              experience_ TEXT,
              employment_ TEXT, 
              total_salary NUMERIC(10,2),
              description TEXT, 
              key_skills_under_desc TEXT,
              skills_in_desc TEXT,
              is_intern BOOLEAN   
              );
          """)
        
        columns = ', '.join(vacancy_1.columns)
        placeholders = ', '.join(['%s'] * len(vacancy_1.columns))
        for _, row in vacancy_1.iterrows():
            cursor.execute(f"INSERT INTO vacancy ({columns})"
                            f"VALUES ({placeholders})"
                            f"ON CONFLICT (id) DO NOTHING", tuple(row))
        conn.commit()
        
        cursor.execute("""
          CREATE TABLE IF NOT EXISTS skills (
              id INTEGER REFERENCES vacancy(id),
              skills_in_desc TEXT
              );
          """)
        
        columns = ', '.join(skills_1.columns)
        placeholders = ', '.join(['%s'] * len(skills_1.columns))
        for _, row in skills_1.iterrows():
            cursor.execute("DELETE FROM skills WHERE id = %s", (row['id'],))
            cursor.execute(f"INSERT INTO skills ({columns})"
                            f"VALUES ({placeholders})", tuple(row))
        conn.commit()
        
        

KeyboardInterrupt: 

In [63]:
DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'postgres',
    'user': 'postgres',
    'password': '123'
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:
        vacancy_info = pd.read_sql_query('''SELECT vacancy.*, emp.name_employer, emp.accredited_it_employer, emp.trusted  FROM vacancy
                 LEFT JOIN employers emp on vacancy.id_employer = emp.id_employer
                 WHERE published_at BETWEEN NOW() - INTERVAL '7 days' and NOW()''', conn)
        vacancy_info.drop(columns=['schedule_work_by_days'], inplace=True)
        vacancy_info.to_csv('vacancy_info.csv', index=False)
        
        skills_info = pd.read_sql_query('''SELECT skills.* FROM skills
                    LEFT JOIN vacancy on skills.id = vacancy.id
                    WHERE published_at BETWEEN NOW() - INTERVAL '7 days' and NOW()''', conn)
        skills_info = skills_info[skills_info['skills_in_desc'].notna()]
        skills_info.to_csv('skills_info.csv', index=False)
        
        conn.commit()
        
        

C:\Users\sekin\AppData\Local\Temp\ipykernel_24416\4125484649.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy_info = pd.read_sql_query('''SELECT vacancy.*, emp.name_employer, emp.accredited_it_employer, emp.trusted  FROM vacancy
C:\Users\sekin\AppData\Local\Temp\ipykernel_24416\4125484649.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  skills_info = pd.read_sql_query('''SELECT skills.* FROM skills


id                0
skills_in_desc    0
dtype: int64

In [55]:
vacancy_info[vacancy_info['id'] == 123509985]

,id,premium,name,has_test,response_letter_required,published_at,apply_alternate_url,alternate_url,from_salary,to_salary,...,experience_,employment_,total_salary,description,key_skills_under_desc,skills_in_desc,is_intern,name_employer,accredited_it_employer,trusted
295,123509985,False,Бизнес-аналитик,False,False,2025-08-01 07:30:28+00:00,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/123509985,90000.0,NaN,...,От 1 года до 3 лет,Проектная работа,78300.0,<p><strong>Образовательный проект (Доп образов...,[],[],False,ОЛДБРУК,false,True


In [61]:
skills_info

,id,skills_in_desc
0,123520648,SQL
1,123520648,Power BI
2,123520648,Excel
3,123464963,Scrum
4,123464963,Agile
...,...,...
2278,122681207,Scrum
2279,123299299,None
2280,123442137,None
2281,123533004,SQL
